<a href="https://colab.research.google.com/github/murmayasa/murmayasa/blob/main/Prediction_Of_Heart_Disease_Using_A_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn.metrics import mean_squared_error
keras.optimizers.Adam

keras.optimizers.adam.Adam

# Mengakses Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
csv_path = '/content/drive/MyDrive/Colab Notebooks/Heart_Disease_Prediction.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Membaca Dataset

In [4]:
data = pd.read_csv(csv_path)

# Memilih Kolom

In [5]:
columns = ['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR',
           'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease']
data = data[columns]

# Memisahkan fitur dan label

In [6]:
X = data.drop('Heart Disease', axis=1)
y = data['Heart Disease']

# Normalisasi fitur menggunakan StandardScaler

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi dataset menjadi data latih dan data uji

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mengubah data menjadi bentuk yang sesuai untuk model RNN

In [55]:
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

label_encoder = LabelEncoder()

# Mengubah nilai dalam kolom target menjadi representasi numerik
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Pembuatan Model

In [56]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[1]), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Fungsi untuk menghitung MAPE

In [ ]:
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compile Model

In [57]:
learning_rate = 0.0001
model.compile(loss='binary_crossentropy',
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 64)                19968     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20,033
Trainable params: 20,033
Non-trainable params: 0
_________________________________________________________________


# Melatih model

In [58]:
model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test))
print("*** Hasil Training ***")
loss, accuracy = model.evaluate(X_test_rnn, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
7/7 [==============================] - 3s 59ms/step - loss: 0.6970 - accuracy: 0.4769 - val_loss: 0.6858 - val_accuracy: 0.5370
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6948 - accuracy: 0.5093 - val_loss: 0.6841 - val_accuracy: 0.5556
Epoch 3/50
7/7 [==============================] - 0s 12ms/step - loss: 0.6926 - accuracy: 0.5324 - val_loss: 0.6824 - val_accuracy: 0.5926
Epoch 4/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6905 - accuracy: 0.5463 - val_loss: 0.6807 - val_accuracy: 0.5741
Epoch 5/50
7/7 [==============================] - 0s 11ms/step - loss: 0.6884 - accuracy: 0.5972 - val_loss: 0.6789 - val_accuracy: 0.6111
Epoch 6/50
7/7 [==============================] - 0s 13ms/step - loss: 0.6863 - accuracy: 0.6111 - val_loss: 0.6772 - val_accuracy: 0.6296
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.6842 - accuracy: 0.6389 - val_loss: 0.6754 - val_accuracy: 0.6481
Epoch 8/50
7/7 [==============

# Evaluasi model

In [68]:
loss, accuracy = model.evaluate(X_test_rnn, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

2/2 [==============================] - 0s 14ms/step - loss: 0.5901 - accuracy: 0.9259
Loss:  0.5901351571083069
Accuracy:  0.9259259104728699


# Tahap Denormalisasi

In [66]:
y_pred_classes = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
y_pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1) * np.sqrt(scaler.var_) + scaler.mean_)
y_test_denormalized = scaler.inverse_transform(y_test.reshape(-1, 1) * np.sqrt(scaler.var_) + scaler.mean_)

# Menghitung RMSE & MAPE

In [67]:
rmse = np.sqrt(np.mean((y_test_denormalized - y_pred_denormalized)**2))
mape = np.mean(np.abs((y_test_denormalized - y_pred_denormalized) / y_test_denormalized)) * 100

print("RMSE:", rmse)
print("MAPE:", mape)

RMSE: 338.35814202626216
MAPE: 12.784256138954293
